In [1]:
!pip install splinter

In [26]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import re
import time
import pandas as pd

In [4]:
#browser = Browser('chrome', executable_path = 'chromedriver',headless='False')
browser = Browser('chrome',executable_path = 'app/chromedriver', headless='False')

# Visit The NASA Mars news site

In [5]:
url = 'https://mars.nasa.gov/news'
browser.visit(url)

In [6]:
browser.is_element_present_by_css('ul.item_list li.slide', wait_time=1)

True

In [7]:
html = browser.html
news_soup = bs(html, 'html.parser')
slide_elem= news_soup.select_one('ul.item_list li.slide')

print(slide_elem)

<li class="slide"><div class="image_and_description_container"><a href="/news/8654/how-nasas-perseverance-mars-team-adjusted-to-work-in-the-time-of-coronavirus/" target="_self"><div class="rollover_description"><div class="rollover_description_inner">Like much of the rest of the world, the Mars rover team is pushing forward with its mission-critical work while putting the health and safety of their colleagues and community first.</div><div class="overlay_arrow"><img alt="More" src="/assets/overlay-arrow.png"/></div></div><div class="list_image"><img alt="Members of NASA's Perseverance rover mission work remotely from home during the coronavirus outbreak." src="/system/news_items/list_view_images/8654_MAIN-IMAGE-PIA23881-320x240.jpg"/></div><div class="bottom_gradient"><div><h3>How NASA's Perseverance Mars Team Adjusted to Work in the Time of Coronavirus </h3></div></div></a><div class="list_text"><div class="list_date">April 21, 2020</div><div class="content_title"><a href="/news/8654/

In [8]:
slide_elem.find('div', class_='content_title')

<div class="content_title"><a href="/news/8654/how-nasas-perseverance-mars-team-adjusted-to-work-in-the-time-of-coronavirus/" target="_self">How NASA's Perseverance Mars Team Adjusted to Work in the Time of Coronavirus </a></div>

In [9]:
news_title = slide_elem.find('div',class_='content_title').get_text()
news_title

"How NASA's Perseverance Mars Team Adjusted to Work in the Time of Coronavirus "

In [10]:
news_p = slide_elem.find('div',class_='article_teaser_body').get_text()
news_p

'Like much of the rest of the world, the Mars rover team is pushing forward with its mission-critical work while putting the health and safety of their colleagues and community first.'

# JPL Space Images Featured Image

In [11]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [12]:
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [13]:
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.find_link_by_partial_text('more info')
more_info_elem.click()

C:\Users\smembaye\AppData\Local\Continuum\anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [14]:
html = browser.html
img_soup = bs(html,'html.parser')

In [15]:
img_url_rel = img_soup.select_one('figure.lede a img').get('src')
img_url_rel

'/spaceimages/images/largesize/PIA18245_hires.jpg'

In [16]:
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18245_hires.jpg'

# Mars Weather

In [17]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

In [18]:
time.sleep(5)

In [19]:
html = browser.html
weather_soup = bs(html, 'html.parser')

In [20]:
mars_weather_tweet = weather_soup.find('div', attrs={'class':'tweet','data-name':'Mars Weather'})

In [21]:
try:
    mars_weather = mars_weather_tweet.find('p', 'tweet-text').get_text()
    mars_weather
except AttributeError:
    pattern = re.compile(r'sol')
    mars_weather = weather_soup.find('span', text=pattern).text
    mars_weather
mars_weather

'InSight sol 501 (2020-04-24) low -93.5ºC (-136.2ºF) high -4.3ºC (24.3ºF)\nwinds from the SW at 5.0 m/s (11.3 mph) gusting to 15.8 m/s (35.3 mph)\npressure at 6.70 hPa'

# Mars Hemispheres

In [22]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [23]:
hemisphere_img_urls = []
links = browser.find_by_css('a.product-item h3')

for i in range(len(links)):
    hemisphere = {}
    
    browser.find_by_css('a.product-item h3')[i].click()
    sample_elem = browser.find_link_by_text('Sample').first
    hemisphere['img_url']  = sample_elem['href']
    hemisphere['title'] = browser.find_by_css('h2.title').text
    hemisphere_img_urls.append(hemisphere)
    
    browser.back()

C:\Users\smembaye\AppData\Local\Continuum\anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:536: FutureWarning: browser.find_link_by_text is deprecated. Use browser.links.find_by_text instead.
  FutureWarning,


In [24]:
hemisphere_img_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

# Mars Facts

In [30]:
df = pd.read_html('https://space-facts.com/mars/')[0]
df.columns = ["Description", 'Value']

In [33]:
df.head()

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [32]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Description</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium 